<a href="https://colab.research.google.com/github/stefanoiervese/DL_Project/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.0/727.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 11.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import os
os.getcwd()
path='/content/drive/MyDrive/Arithmetic/'
data_list=[]
contents=[]
print(os.listdir(path))
for file in os.listdir(path):

   with open(path+file, "r") as file:
    # Leggi il contenuto del file
    content = file.read()
    content=content.split('\n')
    contents= contents+ content
print(len(contents))


['arithmetic__add_or_sub.txt', 'arithmetic__mul.txt', 'arithmetic__div.txt']
3999999


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.getcwd()
path='/content/drive/MyDrive/train-easy/arithmetic__div.txt'
#os.listdir(path)
with open(path, "r") as file:
    # Leggi il contenuto del file
    content = file.read()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import random
import re


data_list = [x for x in contents]
data_list=data_list[:-1]
while '' in data_list:
    data_list.remove('')
len_data=len(data_list)
quest=[]
ans=[]
for i in range(len_data):
  if(i%2==0):
    quest.append(data_list[i])
  else:
    data="# " + data_list[i] + " @"
    ans.append(data)
coppie = list(zip(quest,ans))
random.shuffle(coppie)
quest, ans=zip(*coppie)
l=int(len(quest)/3)
train_q=quest[:2*l]
test_q=quest[2*l:]
train_a=ans[:2*l]
test_a=ans[2*l:]



def crea_vocabolario(frasi):
    vocabolario = set()

    for frase in frasi:
        # Utilizza un'espressione regolare per estrarre parole, numeri e simboli dalla frase
        txt = re.sub(r'(\d+)', lambda x: ' '.join(list(x.group(1))), frase.lower())
        parole = re.findall(r'\b\w+\b|\d+|[^\w\s]', txt)

        # Aggiungi le parole al vocabolario
        vocabolario.update(parole)


    return ['&','#','@','unknown']+list(vocabolario)

v=crea_vocabolario(data_list)

In [ ]:
class Tokenizer:
    def __init__(self, vocab):
        self.vocab = vocab
        self.word_to_id = {word: idx for idx, word in enumerate(vocab)}

    def tokenize(self, text):
        txt = re.sub(r'(\d+)', lambda x: ' '.join(list(x.group(1))), text)
        tokens = re.findall(r'\b\w+\b|\d+|[^\w\s]', txt)



        token_id = []
        unknown_tokens = []

        new_tokens = []
        for token in tokens:
          flag=0
          if '.' in token and token != '.':
            token=token=token.split('.')[0]
            flag=1
          if token in self.word_to_id:
            token_id.append(self.word_to_id[token])
          else:
                token_id.append(self.word_to_id['unknown'])
                unknown_tokens.append(token)
          if(flag==1):
            token_id.append(self.word_to_id['.'])


        if unknown_tokens:

          print("Parole sconosciute:", unknown_tokens)

        return token_id



tokenizer = Tokenizer(v)



In [ ]:
import torch
qt=[]
at=[]
for x in train_q:
  qt.append(torch.tensor(tokenizer.tokenize(x.lower())))
for x in train_a:
  at.append(torch.tensor(tokenizer.tokenize(x.lower())))

qtest=[]
atest=[]
for x in test_q:
  qtest.append(torch.tensor(tokenizer.tokenize(x.lower())))
for x in test_a:
  atest.append(torch.tensor(tokenizer.tokenize(x.lower())))






In [ ]:

import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pack_sequence, pad_sequence


class Dataset(Dataset):
    def __init__(self, questions, answers):
        self.questions = questions
        self.answers = answers

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        return self.questions[idx], self.answers[idx]




#voc_size=100
#src_data = torch.randint(1, voc_size, (3264, 6))
#tg_data=torch.randint(1,voc_size,(3264,8))
#fake_dataset=Dataset(src_data,tg_data)
#fake_loader=DataLoader(fake_dataset,batch_size=32,shuffle=True)

max_length1 = max(len(tensor) for tensor in qt)
max_length2 = max(len(tensor) for tensor in at)
max_length=max(max_length1,max_length2)
qtp = pad_sequence([torch.cat([tensor, torch.zeros(max_length - len(tensor),dtype=torch.int)]) for tensor in qt], batch_first=True)
atp = pad_sequence([torch.cat([tensor, torch.zeros(max_length - len(tensor),dtype=torch.int)]) for tensor in at], batch_first=True)


max_length1 = max(len(tensor) for tensor in qtest)
max_length2 = max(len(tensor) for tensor in atest)
max_length=max(max_length1,max_length2)
qtestp = pad_sequence([torch.cat([tensor, torch.zeros(max_length - len(tensor),dtype=torch.int)]) for tensor in qtest], batch_first=True)
atestp = pad_sequence([torch.cat([tensor, torch.zeros(max_length - len(tensor),dtype=torch.int)]) for tensor in atest], batch_first=True)


batch_size = 64

train_dataset = Dataset(qtp,atp)
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
'''
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,collate_fn=lambda batch: (
    pad_sequence([item[0] for item in batch], batch_first=True),
    pad_sequence([item[1] for item in batch], batch_first=True)
))'''
test_dataset = Dataset(qtestp,atestp)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)


In [ ]:
# Function that computes the accuracy as defined in the "mathematics dataset" paper [1]
def paper_accuracy(predicted_answers, correct_answers):
  num_correct_answers = 0
  end_of_line=2

  for i in range(len(predicted_answers)):

    # Preparing predicted answer
    single_predicted_answer = torch.argmax(predicted_answers[i], 1).tolist()  # vector of shape (answer_max_length) (concatenates the max value for each row)

    if (1 in single_predicted_answer and 2 in single_predicted_answer):
      single_predicted_answer = single_predicted_answer[1:single_predicted_answer.index(2)]  # removing start and end of line char and additional characters
    else:
      continue
    # Preparing correct answer
    single_correct_answer = correct_answers[i].tolist()
    single_correct_answer = single_correct_answer[1:single_correct_answer.index(2)]  # removing start of line, end of line and following characters

    # If the predicted answer and the correct one have the same exact characters, the predicted answer is correct
    print("Predicted")
    print(single_predicted_answer)
    print('Correct')
    print(single_correct_answer)

    if (single_predicted_answer == single_correct_answer):
      num_correct_answers += 1

  return num_correct_answers/len(predicted_answers)


def print_correct(dictionary, batch, res):

  batch_q, batch_a =batch

  for i in range(len(res)):

    index_list = torch.argmax(res[i], 1) # reverse one_hot
    predicted_string = answer_string = question_string = ""

    for j in range(len(batch_q[i])):
      question_string = question_string  + " " + dictionary[batch_q[i][j].item()]

    for j in range(len(index_list)):
      predicted_string = predicted_string + " " + dictionary[index_list[j].item()]

    for j in range(len(batch_a[i])):
      answer_string = answer_string + " " + dictionary[batch_a[i][j].item()]

    print(dictionary)

    print("QUESTION " + question_string + "\n", "PREDICTED " + predicted_string + "\n","CORRECT " + answer_string + "\n")



In [ ]:
from torch.nn.modules import dropout
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import random


class LSTM(pl.LightningModule):
  def __init__(self, dict_size, sizes, dictionary, teacher_forcing_ratio: float = 0.5):
    super().__init__()

    self.hidden_units = 2048
    self.question_max_length = sizes[0]
    self.answer_max_length = sizes[1]
    self.dict_size = dict_size
    self.teacher_forcing_ratio = teacher_forcing_ratio
    self.dictionary = dictionary

    # Initializing used layers
    self.lstm = nn.LSTM(self.dict_size, self.hidden_units)
    self.linear_layer = nn.Linear(self.hidden_units, dict_size, bias=False)  # used to return vectors of shape (dict_size)

  def forward(self, batch):
    batch_q, batch_a = batch

    batch_size = len(batch_q)
    batch_questions = batch_q  # shape (batch_size, question_max_length)
    batch_questions = torch.transpose(batch_questions, 0, 1)  # shape (question_max_length, batch_size)
    batch_questions = F.one_hot(batch_questions, self.dict_size)  # shape (question_max_length, batch_size, dict_size)
    batch_questions = batch_questions.float()

    if self.training:
      batch_answers = batch_a  # shape (batch_size, answer_max_length)
      batch_answers = torch.transpose(batch_answers, 0, 1)  # shape (answer_max_length, batch_size)
      batch_answers = F.one_hot(batch_answers, self.dict_size)  # shape (answer_max_length, batch_size, dict_size)
      batch_answers = batch_answers.float()

    # Initializing hidden_state and cell_state used by the lstm cell. In our case D = 1 (because it is not bidirectional) and num_layer = 1
    hidden_state = torch.zeros(1, batch_size, self.hidden_units, requires_grad=True, dtype=torch.float).to(device)  # shape (D*num_layers, batch_size, H_out)
    cell_state = torch.zeros(1, batch_size, self.hidden_units, requires_grad=True, dtype=torch.float).to(device)  # shape (D*num_layers, batch_size, H_cell)

    # Initializing result tensor
    result = torch.empty(self.answer_max_length, batch_size, self.dict_size).to(device)  # shape (answer_max_length, batch_size, dict_size)

    self.lstm.flatten_parameters()

    # input shape (1, batch_size, dict_size)
    # output shape (1, batch_size, hidden_size)
    for i in range(self.question_max_length):
      output, (hidden_state, cell_state) = self.lstm(batch_questions[i].unsqueeze(0), (hidden_state, cell_state))


    for i in range(self.answer_max_length):

      result_temp = self.linear_layer(output[0])  # shape (batch_size, dict_size)
      result[i] = result_temp

      if self.training:
        teacher_force = random.random() < self.teacher_forcing_ratio  # true or false with prob 'teacher_forcing_ratio'

        if teacher_force:  # use correct previous char
          input = batch_answers[i].unsqueeze(0)  # shape (1, batch_size, dict_size)
        else:  # use previous predicted char
          input = F.one_hot(torch.argmax(result_temp, 1), self.dict_size).unsqueeze(0).float().to(device)  # shape (1, batch_size, dict_size)

      else:
        if i == 0:
          input = F.one_hot(torch.tensor([1]), self.dict_size).repeat(batch_size, 1).unsqueeze(0).float().to(device)  # shape (1, batch_size, dict_size)
        else:
          input = F.one_hot(torch.argmax(result_temp, 1), self.dict_size).unsqueeze(0).float().to(device)  # shape (1, batch_size, dict_size)

      output, (hidden_state, cell_state) = self.lstm(input, (hidden_state, cell_state))  # shape (1, batch_size, hidden_size)
    return torch.transpose((result), 0, 1)  # shape (batch_size, answer_max_length, dict_size)

  def training_step(self, batch, _):
    batch_q, batch_a = batch
    print(batch_q.shape)
    # Preparing inputs
    batch_answers = batch_a  # shape (batch_size, answer_max_length)
    batch_answers = batch_answers.flatten(0, 1)  # shape (batch_size * answer_max_length)

    # Computing prediction and loss

    pred = self(batch).flatten(0, 1)  # shape (batch_size * answer_max_length, dict_size)
    loss = F.cross_entropy(pred, batch_answers, ignore_index=0)

    print(loss)

    return loss

  def validation_step(self, batch, _):
    batch_q, batch_a = batch
    batch_answers = batch_a  # shape (batch_size, answer_max_length)

    # Computing prediction and accuracy
    pred = self.predict(batch)  # shape (batch_size, answer_max_length, dict_size)
    accuracy = paper_accuracy(pred, batch_answers)

    self.log("val_tot_accuracy", accuracy)
    return accuracy

  def test_step(self, batch, batch_idx):

    batch_q, batch_a = batch

    # Computing prediction and accuracy
    pred = self(batch)  # shape (batch_size, answer_max_length, dict_size)
    accuracy = paper_accuracy(pred, batch_a)

    print_correct(self.dictionary, batch, pred)

    print(accuracy)
    acc.append(accuracy)
    return accuracy

  def predict(self, questions):
    return self(questions)

  def print_predict(self, questions):
    pred = self.predict(questions.copy())
    return

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=1e-4, betas=(0.9, 0.995), eps=1e-9)

In [ ]:
qt_max_len= 22
at_max_len = 22
sizes=[qt_max_len, at_max_len]
acc=[]
losses=[]
net = LSTM(len(v), sizes, v)
trainer= pl.Trainer(max_epochs=1)

trainer.fit(net, train_loader)



INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:71: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type   | Params
----------------------------------------
0 | lstm         | LSTM   | 17.2 M
1 | linear_layer | Linear | 102 K 
----------------------------------------
17.3 M    Trainable params
0         Non-trainable params
17.3 M    Total params
69.222    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Output streaming troncato alle ultime 5000 righe.
torch.Size([64, 22])
tensor(0.1854, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([64, 22])
tensor(0.1501, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([64, 22])
tensor(0.2059, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([64, 22])
tensor(0.0946, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([64, 22])
tensor(0.1084, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([64, 22])
tensor(0.1532, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([64, 22])
tensor(0.1158, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([64, 22])
tensor(0.1555, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([64, 22])
tensor(0.1299, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([64, 22])
tensor(0.1197, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([64, 22])
tensor(0.2042, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([64, 22])
tensor(0.1350, device='cuda:0', grad_f

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
model_path='/content/drive/MyDrive/model/modello.pth'

torch.save(net.state_dict(), model_path)

In [ ]:
model_path='/content/drive/MyDrive/model/modello.pth'
net.load_state_dict(torch.load(model_path))
net.eval()

trainer.test(net,test_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:490: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

Output streaming troncato alle ultime 5000 righe.
QUESTION  calculate - 3 6 4 5 divided by - 2 4 3 . & & & & & & & & &
 PREDICTED  # 1 5 @ @ 9 @ @ @ @ @ @ 2 5 3 . 0 4 9 4 9 4
 CORRECT  # 1 5 @ & & & & & & & & & & & & & & & & & &

0.890625
Predicted
[13, 25]
Correct
[13, 25]
Predicted
[15, 25]
Correct
[15, 25]
Predicted
[25, 49, 13, 46, 27]
Correct
[25, 49, 13, 46, 27]
Predicted
[15, 27, 23, 34, 33, 33, 22, 23, 13, 23]
Correct
[15, 27, 23, 34, 33, 33, 22, 23, 13, 23]
Predicted
[15, 10, 13, 34, 33, 33, 22, 10]
Correct
[15, 10, 13, 34, 33, 33, 22, 10]
Predicted
[15, 31, 34, 20]
Correct
[15, 31, 34, 20]
Predicted
[25, 46, 34, 25, 13, 13, 13]
Correct
[25, 46, 34, 25, 13, 13, 13]
Predicted
[23]
Correct
[23]
Predicted
[10]
Correct
[10]
Predicted
[20, 25, 23]
Correct
[20, 25, 23]
Predicted
[27, 34, 46, 22, 25, 20]
Correct
[27, 34, 46, 22, 25, 20]
Predicted
[25, 33, 34, 31]
Correct
[25, 33, 34, 31]
Predicted
[46]
Correct
[46]
Predicted
[15, 25, 27, 22]
Correct
[15, 25, 27, 22]
Predicted
[13]
Co

[{}]

In [ ]:
sum(acc)/len(acc)

0.8090648956376253

In [ ]:
print(losses)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses)
plt.show()

NameError: ignored